# Web scraping the sites that contain site maps

This notebook aims to gather all the data from multiple that contain site maps. The target is to get as many product websites from different pages with as many different furniture types as possible.

In [1]:
# importing libraries

import csv 
import operator

import requests
from tqdm import tqdm 
from bs4 import BeautifulSoup

from concurrent.futures import ThreadPoolExecutor
import concurrent

from urllib.parse import urlparse
from urllib.parse import urljoin

### Methods for getting the base url and for checking if the website has a sitemap

In [2]:



SITEMAP_PATHS = [
    "sitemap.xml",
    "sitemap_index.xml",
    ".sitemap.xml",
    "sitemap/sitemap.xml",
    "sitemap_index/sitemap.xml"
]

def get_base_url(url):
    try:
        parsed_url = urlparse(url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
        return base_url
    except Exception as e:
        # print(f"Error parsing URL {url}: {e}")
        return None

def check_sitemap(base_url):
    """
    Check if the given base URL has a sitemap in one of the common paths.
    Returns the URL of the sitemap if found, otherwise None.
    """
    for sitemap_path in SITEMAP_PATHS:
        sitemap_url = urljoin(base_url, sitemap_path)
        try:
            response = requests.head(sitemap_url, timeout=10)
            # Check if the URL exists and returns a successful status code (200)
            if 200 <= response.status_code < 300:
                return sitemap_url
        except requests.RequestException as e:
            # print(f"Error checking {sitemap_url}: {e}")
            continue
    return None

def check_sitemap_concurrently(base_urls):
    """
    Checks sitemaps for a list of base URLs concurrently.
    Returns a list of results with the base URL and sitemap URL.
    """
    results = []
    with ThreadPoolExecutor(max_workers=16) as executor:
        futures = {executor.submit(check_sitemap, base_url): base_url for base_url in base_urls}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Checking sitemaps"):
            base_url = futures[future]
            sitemap_url = future.result()
            if sitemap_url:
                results.append([base_url, "Sitemap found", sitemap_url])
            else:
                results.append([base_url, "No sitemap found", ""])
    return results

### Filtering out the websites that are not accessible that have a site map

In [3]:
# reading the csv file and storing the links in the links list
links = []
with open('furniture stores pages.csv', mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        try:
            link = get_base_url(row[0])
            links.append(link)
        except Exception as e:
            continue
        

# Check sitemaps concurrently
sitemap_results = check_sitemap_concurrently(links)

# Write the results to the output CSV file
output_csv = "sitemap_results.csv"
with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["URL", "Status", "Sitemap URL"])  # Header row
    csvwriter.writerows(sitemap_results)  # Write all results

print(f"Results saved to {output_csv}.")

Checking sitemaps: 100%|██████████| 705/705 [02:23<00:00,  4.91it/s]

Results saved to sitemap_results.csv.


### Getting the sitemaps from the csv file

In [3]:
# getting only the sitemaps from the csv file
sitemaps = []
with open('sitemap_results.csv', mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if row[1] != 'No sitemap found':
            sitemaps.append(row[2])

print(sitemaps[1:], len(sitemaps))

['https://cane-line.co.uk/sitemap.xml', 'https://edenliving.online/sitemap.xml', 'https://dunlin.com.au/sitemap.xml', 'https://vastinterior.com.au/sitemap.xml', 'https://hemisphereliving.com.au/sitemap.xml', 'https://www.factorybuys.com.au/sitemap.xml', 'https://dhfonline.com/sitemap.xml', 'https://www.tandemarbor.com/sitemap.xml', 'https://www.perchfurniture.com/sitemap.xml', 'https://modshop1.com/sitemap.xml', 'https://www.ourfurniturewarehouse.com.au/sitemap.xml', 'https://www.hudsonfurniture.com.au/sitemap.xml', 'https://www.scandesign.com/sitemap.xml', 'https://www.sofamania.com/sitemap.xml', 'https://www.fentonandfenton.com.au/sitemap.xml', 'https://4-chairs.com/sitemap.xml', 'https://www.knoll.com/sitemap.xml', 'https://acmeshelving.com/sitemap.xml', 'https://claytongrayhome.com/sitemap.xml', 'https://www.do-shop.com/sitemap.xml', 'https://premiumpatio.com.au/sitemap.xml', 'https://www.theinside.com/sitemap.xml', 'https://www.kmpfurniture.com/sitemap.xml', 'https://www.jseitz.co

### Methods for getting the links from the sitemaps

In [4]:
def get_data(url):
    response = requests.get(url)
    return response.text

def is_valid_product_link(url):
    # Exclude common unwanted patterns
    unwanted_patterns = ['.jpg', '.jpeg', '.png', '.gif', '.svg', '.css', '.js', '.ico', 'tel:', 'mailto:', '#']
    for pattern in unwanted_patterns:
        if pattern in url:
            return False
    # Only accept URLs that contain "collections" or "products"
    if '/products/' in url: # or  'collections' in url: - right now I will only focus on the products path
        return True
    return False

def is_desired_site_map_link(url): #  
    # Only accept URLs that contain "sitemap"
    if 'sitemap_products_1.xml' in url: # most website have this in the url that I am looking for
        return True
    return False


dict_href_links = {}

def get_links_from_sitemap(website_link): # modified version from the one in the other notebook
    # Set the base of the URL depending on whether "collections" or "products" is in the link
    website_origin = website_link
    sitemap_index = website_link.find('/sitemap')
    if sitemap_index == -1:
        sitemap_index = website_link.find('/.sitemap')
    if sitemap_index != -1:
        website_origin = website_link[:sitemap_index + 1] 

    html_data = get_data(website_link)
    soup = BeautifulSoup(html_data, "html.parser")
    list_links = []

    for link in soup.find_all("loc"): # this contains the links inside xml files
        link = link.text
        # Filter out invalid links (non-product/collection pages)
        
        if not is_valid_product_link(link) and not is_desired_site_map_link(link): 
            continue
        
        link_to_append = None

        # Handle absolute URLs that start with the origin
        if link.startswith(str(website_origin)):
            link_to_append = link
        
        # Handle relative URLs that start with "/"
        elif link.startswith("/"):
            #print(href)
            link_with_www = website_origin + link[1:]
            #print("adjusted link =", link_with_www)
            link_to_append = link_with_www
        

        
        # If link_to_append is not None, check if it's already in dict_href_links and if it's accessible
        if link_to_append is not None:
            if link_to_append not in dict_href_links: #  and check_website(link_to_append) - I will not check the links here, I will check them after I get all the links
                dict_href_links[link_to_append] = None  # Mark it as seen

                list_links.append(link_to_append)

    # Convert list of links to a dictionary with "Not-checked" as the default value for each
    dict_links = dict.fromkeys(list_links, "Not-checked")
    return dict_links

def get_subpage_links(l, max_depth=3, current_depth=0, write_frequency=500, csv_filename="link_data.csv"):
    processed_links_count = 0
    
    if current_depth >= max_depth:
        return l

    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = {executor.submit(get_links_from_sitemap, link): link for link in l if l[link] == "Not-checked"}
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing subpage links"):
            link = futures[future]
            try:
                dict_links_subpages = future.result()
            except Exception as e:
                print(f"Error fetching {link}: {e}")
                continue
            l[link] = "Checked"
            l.update(dict_links_subpages)

            processed_links_count += 1
            
            # Write to file every 'write_frequency' processed links
            if processed_links_count >= write_frequency:
                write_links_to_csv(l, csv_filename)
                processed_links_count = 0  # Reset the counter

    # Recursively call the function for the next depth level
    return get_subpage_links(l, max_depth, current_depth + 1, write_frequency, csv_filename)

def write_links_to_csv(links_dict, csv_filename):
    """Writes the current state of the links dictionary to a CSV file."""
    with open(csv_filename, "w", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for link in links_dict.keys():
            csvwriter.writerow([link])

    print(f"Links saved to {csv_filename}.")


### Getting the links from the sitemaps


In [ ]:
# we gather all the links from some pages

# we test with the first page inside the final_links csv

websites = sitemaps[1:]  # exclude the first row
# create dictionary of website
dict_links = { website : "Not-checked" for website in websites }

counter, counter2 = None, 0
csv_filename = "link_data.csv"

while counter != 0:
    counter2 += 1
    dict_links2 = get_subpage_links(dict_links, csv_filename=csv_filename)
    counter = operator.countOf(dict_links2.values(), "Not-checked")  # Number of "Not-checked" links
    
    # Print some statements for debugging
    print("")
    print("THIS IS LOOP ITERATION NUMBER", counter2)
    print("LENGTH OF DICTIONARY WITH LINKS =", len(dict_links2))
    print("NUMBER OF 'Not-checked' LINKS = ", counter)
    print("")
    
    dict_links = dict_links2
    

write_links_to_csv(dict_links, csv_filename)

print("Links saved to link_data.csv.")

Processing subpage links:   0%|          | 0/385 [00:00<?, ?it/s]C:\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Processing subpage links:   2%|▏         | 533/32551 [00:17<05:45, 92.58it/s] 

Links saved to link_data.csv.


Processing subpage links:   3%|▎         | 1034/32551 [00:36<10:40, 49.20it/s] 

Links saved to link_data.csv.


Processing subpage links:   5%|▍         | 1500/32551 [01:15<2:32:59,  3.38it/s]

...

Processing subpage links:   6%|▌         | 2000/32551 [01:36<56:18,  9.04it/s]  

Links saved to link_data.csv.


Processing subpage links:   8%|▊         | 2551/32551 [01:56<13:31, 36.99it/s]  

Links saved to link_data.csv.


Processing subpage links:   8%|▊         | 2707/32551 [02:02<22:40, 21.94it/s]